In [1]:
%load_ext autoreload
%autoreload 2

## DB Setup

We assume you already have a sqlite database ready.

In [2]:
import pandas as pd
import sqlite3

#### database setup, uncomment if database doesn't exist

download data from https://www.kaggle.com/datasets/ranadeep/credit-risk-dataset

In [3]:
# df = pd.read_csv('data/loan.csv')
# conn = sqlite3.connect('loan.db')
# df.to_sql('loan', conn, index=False)

In [4]:
DATABASE = "loan.db"
TABLES = []  # list of tables to load or [] to load all tables

In [5]:
from db_connectors import SQLiteConnector
from prompt_formatters import RajkumarFormatter

# Get the connector and formatter
sqlite_connector = SQLiteConnector(
    database_path=DATABASE
)
sqlite_connector.connect()
if len(TABLES) <= 0:
    TABLES.extend(sqlite_connector.get_tables())

print(f"Loading tables: {TABLES}")

db_schema = [sqlite_connector.get_schema(table) for table in TABLES]
formatter = RajkumarFormatter(db_schema)

Loading tables: ['loan']


## Model Setup

In a separate screen or window, first install [Manifest](https://github.com/HazyResearch/manifest)
```bash
pip install manifest-ml\[all\]
```

Then run
```bash
python3 -m manifest.api.app \
    --model_type huggingface \
    --model_generation_type text-generation \
    --model_name_or_path NumbersStation/nsql-6B \
    --device 0
```

If successful, you will see an output like
```bash
* Running on http://127.0.0.1:5000
```

In [7]:
from manifest import Manifest

manifest_client = Manifest(client_name="huggingface", client_connection="http://127.0.0.1:5000")

def get_sql(instruction: str, max_tokens: int = 300) -> str:
    prompt = formatter.format_prompt(instruction)
    res = manifest_client.run(prompt, max_tokens=max_tokens)
    return formatter.format_model_output(res)

In [44]:
questions = [
    "What is the member_id for the user who has the highest risk?",         # illogical question
    "How many rows are in the loan database?",                              # basic 
    "What is the breakdown of the grade column?",                           # basic
                                                                            # basic longer
    "Count the number of rows where the loan status is any of the following ['Charged Off', 'Default', 'Does not meet the credit policy. Status:Charged Off', 'In Grace Period', 'Late (31-120 days)', 'Late (16-30 days)']",
                                                                            # limitation: alter table
    "Alter table to add a column titled 'class' that is 1 if loan_status is any of the following ['Charged Off', 'Default', 'Does not meet the credit policy. Status:Charged Off', 'In Grace Period', 'Late (31-120 days)', 'Late (16-30 days)'] and 0 otherwise.",
    "How many loans where risky is 1 are there?",                           # basic
    "A loan is risky when risky =  1. Count the risky loans, grouped by term length.",
    "What is the maximum loan amount?", # TODO: top ten for some filter
                                        # TODO: split 
    "Split the original table based on term length?"
]

In [45]:
sql = get_sql(questions[8])
print(sql)

SELECT term, COUNT(*) FROM loan GROUP BY term;


In [46]:
print(sqlite_connector.run_sql_as_df(sql))

         term  COUNT(*)
0   36 months    621125
1   60 months    266254


#### TODO: 
1. Research architecture of NSQL.
2. How did nsql perform in benchmarks?
3. How was nsql fine-tuned?
4. Transfer sqlite to db2 for better scaling.

Add Riskiness to Database

In [28]:
# risky_statuses = ['Charged Off', 'Default', 'Does not meet the credit policy. Status:Charged Off', 'In Grace Period',
#                       'Late (31-120 days)', 'Late (16-30 days)']

# df['risky'] = df.loan_status.apply(lambda x: int(x in risky_statuses))

# df.to_sql('loan', conn, if_exists='replace', index=False)